<a href="https://colab.research.google.com/github/MIAShield/MIAShield/blob/main/Evaluation_of_MIAShield_(Cifar10_and_100).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


import glob
import itertools
import collections
#import fiftyone as fo
#import fiftyone.zoo as foz

from PIL import Image
import cv2
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import torch
import imagehash
import hashlib


#importing libraries
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# general imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from scipy import special

# tensorflow imports
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D,Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

In [2]:
# tensorflow-privacy libraries
from tensorflow_privacy.privacy.membership_inference_attack import membership_inference_attack as mia
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import AttackType

C:\Users\neelk\anaconda3\envs\solo2\lib\site-packages\tensorflow_privacy\privacy\membership_inference_attack\__init__.py:18: UserWarning: 
Membership inference attack sources were moved. Please replace
import tensorflow_privacy.privacy.membership_inference_attack

with
import tensorflow_privacy.privacy.privacy_tests.membership_inference_attack
  warnings.warn(


# EO IS HERE #
Initialization

In [3]:
dataset='Cifar10'            #dataset, 'Cifar10'/'Cifar100'
EO='EO3_1'                     #EO: 'EO2','EO3', 'EO4', 'EO5'
batch_size=128

In [4]:

  #define all the variables
#cifar10
shape = (32, 32, 3)
num_class=10
learning_rate=.01
epochs=60
M=5



In [5]:
def load_cifar10():
  """Loads cifar10-Dataset and preprocesses to combine training and test data."""
  
  # load the existing CIFAR10 dataset that comes in form of traing + test data and labels
  train, test = tf.keras.datasets.cifar10.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  # scale the images from color values 0-255 to numbers from 0-1 to help the training process
  train_data1 = np.array(train_data, dtype=np.float32) / 255
  test_data1 = np.array(test_data, dtype=np.float32) / 255

  # cifar10 labels come one-hot encoded, there
  train_labels = train_labels.flatten()
  test_labels = test_labels.flatten()

  return train_data1, train_labels, test_data1, test_labels, train_data, train_labels, test_data, test_labels 

In [6]:
train_data, train_labels, test_data, test_labels,trainX1,trainY1, testX1,testY1 = load_cifar10()  

In [7]:
train_y=to_categorical(train_labels)
test_y=to_categorical(test_labels)

In [8]:
def AlexnetModel(input_shape,num_classes):
    model = Sequential()
# model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', padding='same', input_shape=(img_height, img_width, channel,)))
# for original Alexnet
    model.add(Conv2D(48, (3,3), strides=(2,2), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
# Local Response normalization for Original Alexnet
    model.add(BatchNormalization())

    model.add(Conv2D(96, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
    model.add(BatchNormalization())

    model.add(Conv2D(192, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(192, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

Common Functions

In [9]:
def confidence(my_list):
  import numpy as np
  my_array=np.array(my_list)
  p=np.zeros(my_array.shape)
  b=my_array.max(-1)
  condition = my_array == b[..., np.newaxis]
  c = np.where(condition, 1, 0)
  final=np.multiply(c, my_array)
  #my_sum=np.sum(final,axis=0)
  labels=np.argmax(final, axis=1)
  return final

In [10]:
def label(my_list):
  import numpy as np
  my_array=np.array(my_list)
  p=np.zeros(my_array.shape)
  b=my_array.max(-1)
  condition = my_array == b[..., np.newaxis]
  c = np.where(condition, 1, 0)
  final=np.multiply(c, my_array)
  #my_sum=np.sum(final,axis=0)
  labels=np.argmax(final, axis=1)
  return labels

Non-Private Model_training

In [11]:
# make the neural network model with the function specified above.
# one model is supposed to train for 10, one for 50 epochs
model_np1 = AlexnetModel(shape,num_class)
model_np1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 16, 16, 48)        1344      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 8, 48)         0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 48)         192       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 96)          41568     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 3, 96)         0         
 2D)                                                             
                                                        

In [12]:
# specify parameters
#learning_rate=.005
optimizer = tf.keras.optimizers.SGD(learning_rate=.01)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# compile the model
model_np1.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [13]:
train=train_data[0:5000]
trainX=trainX1[0:5000]
test=test_data[5000:10000]
testX=testX1[5000:10000]

In [14]:
# stop!!

Training MIAShield Models 

# Splitting Reduced (/255) image dataset as x_train_split%s #

In [14]:
#split datasets
#from imblearn.over_sampling import RandomOverSampler

#ros = RandomOverSampler()

#X_ros, y_ros = ros.fit_resample(x_train_split0, trainy_split0)
# split x_train to 5 disjoint datasets
#store each dataset variable name is the list Xtrain
from sklearn.model_selection import StratifiedKFold
X=train_data
Y=train_labels
for i in range(0,M):
  K=M-i
  if(K>=2):
    train=X
    test=Y
    skf = StratifiedKFold(n_splits=K)
    skf.get_n_splits(X, Y)
    #print(skf)
    for train_index, test_index in skf.split(X, Y):
       #print(X.shape)
       X,globals()['x_train_split%s' % i]=train[train_index], train[test_index]
       Y,globals()['trainy_split%s' % i]=test[train_index], test[test_index]
       y_hot=globals()['trainy_split%s' % i]
       y_hot=to_categorical(y_hot)
       globals()['y_train_split%s' % i]=y_hot
       #X=train
       #Y=test
  elif K==1:
    globals()['x_train_split%s' % i]=X
    globals()['trainy_split%s' % i]=Y
    y_hot=to_categorical(Y)
    globals()['y_train_split%s' % i]=y_hot


In [15]:
model_0 = keras.models.load_model('0.h5')

model_1 = keras.models.load_model('1.h5')

model_2 = keras.models.load_model('2.h5')

model_3 = keras.models.load_model('3.h5')

model_4 = keras.models.load_model('4.h5')

model_np = keras.models.load_model('np.h5')


In [16]:
#calculate accuracy/utility test acc
print('\n ACCURACY SCORES \n')
print('\n Model 1 \n')
prb=model_0.predict(test_data[0:5000])
test_labeld_result=np.argmax(prb,axis=1)
print(accuracy_score(test_labels[0:5000],test_labeld_result))
#prob_testd

print('\n Model 2 \n')
#calculate accuracy/utility test acc
prb=model_1.predict(test_data[0:5000])
test_labeld_result=np.argmax(prb,axis=1)
print(accuracy_score(test_labels[0:5000],test_labeld_result))
#prob_testd


print('\n Model 3 \n')
#calculate accuracy/utility test acc
prb=model_2.predict(test_data[0:5000])
test_labeld_result=np.argmax(prb,axis=1)
print(accuracy_score(test_labels[0:5000],test_labeld_result))
#prob_testd


print('\n Model 4 \n')
#calculate accuracy/utility test acc
prb=model_3.predict(test_data[0:5000])
test_labeld_result=np.argmax(prb,axis=1)
print(accuracy_score(test_labels[0:5000],test_labeld_result))
#prob_testd


print('\n Model 5 \n')
#calculate accuracy/utility test acc
prb=model_4.predict(test_data[0:5000])
test_labeld_result=np.argmax(prb,axis=1)
print(accuracy_score(test_labels[0:5000],test_labeld_result))
#prob_testd


print('\n Model NP \n')
#calculate accuracy/utility test acc
prb=model_np.predict(test_data[0:5000])
test_labeld_result=np.argmax(prb,axis=1)
print(accuracy_score(test_labels[0:5000],test_labeld_result))
#prob_testd


 ACCURACY SCORES 


 Model 1 

157/157 [==============================] - 1s 6ms/step
0.6662

 Model 2 

157/157 [==============================] - 1s 6ms/step
0.6496

 Model 3 

157/157 [==============================] - 1s 6ms/step
0.6642

 Model 4 

157/157 [==============================] - 1s 6ms/step
0.6104

 Model 5 

157/157 [==============================] - 1s 6ms/step
0.6048

 Model NP 

157/157 [==============================] - 1s 7ms/step
0.7258


# Splitting original image dataset as 'x_traind%s' #

In [17]:
# from imblearn.over_sampling import RandomOverSampler

# ros = RandomOverSampler()

# X_ros, y_ros = ros.fit_resample(x_train_split0, trainy_split0)
# split x_train to 5 disjoint datasets
# store each dataset variable name is the list Xtrain
from sklearn.model_selection import StratifiedKFold
X=trainX1
Y=trainY1
for i in range(0,M):
  K=M-i
  if(K>=2):
    train=X
    test=Y
    skf = StratifiedKFold(n_splits=K)
    print(skf.get_n_splits(X, Y))
    for train_index, test_index in skf.split(X, Y):
       X,globals()['x_traind%s' % i]=train[train_index], train[test_index]
    #print(X.shape)
       Y,globals()['y_traind%s' % i]=test[train_index], test[test_index]
  elif K==1:
    globals()['x_traind%s' % i]=X
    globals()['y_traind%s' % i]=Y

5
4
3
2


In [19]:
# stop!!!

Exclusion Oracle-2: Hash

Exclusion Oracle-3: ImageHash

In [18]:
def convert_Imhash(trainX1):
  lenth=len(trainX1)
  hashid=np.zeros(lenth)
  hash=[]
  for id in range(lenth):
    hashid[id]=id
    #covert in hashes
    im_array=Image.fromarray(np.uint8(trainX1[id])).convert('RGB')
    #im_array=Image.fromarray(trainX1[id].astype('uint8'), 'RGB')
    #im_array=trainX1[id]
    hash1 = imagehash.phash(im_array)
    #print(type(hash1))
    #hash1=str(hash1)
    #hash1=int(hash1,16)
    #print(type(hash1))
    #print(np.array(hash1))
    hash.append(hash1)
  return np.array(hash)

In [21]:
# import cv2    
# from sklearn import decomposition    
# import matplotlib.pyplot as plt 
# import numpy as np   

# img = X_train  
# vmax = max(img.max(), -img.min())

# fig, (ax, ax2)  =plt.subplots(ncols=2)    
# ax.imshow(img, cmap=plt.cm.gray, interpolation = 'nearest',vmin=-vmax,vmax=vmax)

# n_components = 20

# estimator = decomposition.NMF(n_components = n_components, init = 'random', tol=5e-3)    
# W = estimator.fit_transform(img)
# H = estimator.components_

# def circle_points(resolution, center, radius):
#     """
#     Generate points which define a circle on an image.Centre refers to the centre of the circle
#     """   
#     radians = np.linspace(0, 2*np.pi, resolution)
#     c = center[1] + radius*np.cos(radians)#polar co-ordinates
#     r = center[0] + radius*np.sin(radians)
    
#     return np.array([c, r]).T
# # Exclude last point because a closed path should not have duplicate points

# def get_ring_strip(k=0, m=4):
    
#     for i in range(m):
#         if k==0:            
#             points = circle_points(200, [80, 250], 80)[:-1]
# # img = cv2.imread('img1.jpg')
# # imgYCC = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
# snake = seg.active_contour(image_gray, points,alpha=0.06,beta=0.3)


# from scipy.fftpack import dct, idct

# # implement 2D DCT
# def dct2(a):
#     return dct(dct(a.T, norm='ortho').T, norm='ortho')



In [19]:
def hamming_dist(h1,h2):
  n=len(h2)
  d=np.zeros(n)
  for i in range(n):
#     print(h1)
#     print('h2 \n', h2[i])
    d[i]=h1-h2[i]
  return d

In [20]:
#check which model contain the example
def model_index_EO3(d1,d2,M):
  h1=convert_Imhash(d2)
  for i1 in range(M):
    traind=globals()['x_hash%s' % i1]
    #traind=shuffle(traind1)
    #traind=np.sort(traind)
    
    d=hamming_dist(h1,traind)
    member=np.count_nonzero(d < 14)
    #print(member)
    if (member>=1):
      index=i1
      break
  if (member==0):
    index=M
  return index

# CHANGE HERE!!! #

hamming distance and threshold distance

# EO3_1 AHASH function #

In [24]:
def convert_Imhash1(trainX1):
  lenth=len(trainX1)
  hashid=np.zeros(lenth)
  hash=[]
  for id in range(lenth):
    hashid[id]=id

    im_array=Image.fromarray(trainX1[id].astype('uint8'), 'RGB')

    hash1 = imagehash.average_hash(im_array)

    hash.append(hash1)
  return np.array(hash)


In [25]:
#check which model contain the example
def model_index_EO3_1(d1,d2,M):
    
  h1=convert_Imhash1(d2)
  for i1 in range(M):
    traind=globals()['x_hash%s' % i1]

    
    d=hamming_dist(h1,traind)
    member=np.count_nonzero(d < 14)

    if (member>=1):
      index=i1
      break
  if (member==0):
    index=M
  return index

# EO3_2 AHASH+ DHASH function #


In [26]:
def convert_Imhash2(trainX1):
  lenth=len(trainX1)
  hashid=np.zeros(lenth)
  hash=[]
  for id in range(lenth):
    hashid[id]=id

    im_array=Image.fromarray(trainX1[id].astype('uint8'), 'RGB')

    hash1 = imagehash.average_hash(im_array)

    hash.append(hash1)
  return np.array(hash)


In [27]:
def dhashAfterAhash(data, d2 ):
    
#     print(d2.shape)
    d2_array=Image.fromarray(d2[0].astype('uint8'), 'RGB')
    hackerImHash = imagehash.dhash(d2_array)
    lenth=len(data)
    hashid=np.zeros(lenth)
    hashD=[]
    
    for id in range(lenth):
        hashid[id]=id

        im_array=Image.fromarray(data[id].astype('uint8'), 'RGB')
        hash1 = imagehash.dhash(im_array)
        hashD.append(hash1)
        
        
    d=hamming_dist(hackerImHash, np.array(hashD))
    member=np.count_nonzero(d < 16)
#     print('\n member 2 \n', member)
    return member


In [28]:
#check which model contain the example
def model_index_EO3_2(d1,d2,M):
  
  h1=convert_Imhash2(d2)
  for i1 in range(M):
    traind=globals()['x_hash%s' % i1]
    
    d=hamming_dist(h1,traind)
    member=np.count_nonzero(d < 14)
    
    Fmember =0
    if (member>=1):
        members = d < 14
#         print('member 1 \n ', member)
        trainx=globals()['x_traind%s' % i1]
#         print(members)
        Fmember = dhashAfterAhash(trainx[members,:], d2)
        if Fmember >=1 :
            index=i1
            break
            
  if (Fmember==0):
    index=M
  return index

# EO3_3 BHASH function #


In [29]:
from perception import hashers

In [69]:
def hamming_dist_2(h1,h2):
  n=len(h2)
  d=np.zeros(n)
  for i in range(n):
#     print(h1)
#     print('h2 \n', h2[i])
    hasher = hashers.BlockMean()
    hashdis = hasher.compute_distance(h1, h2[i])
    d[i]=hashdis
  return d

In [70]:
def convert_Imhash3(trainX1):
  lenth=len(trainX1)
  hashid=np.zeros(lenth)
  hash=[]
  for id in range(lenth):
    hashid[id]=id

    im_array=Image.fromarray(trainX1[id].astype('uint8'), 'RGB')
    hasher = hashers.BlockMean()
    hash1 = hasher.compute(im_array)

    hash.append(hash1)
  return np.array(hash)


In [71]:
#check which model contain the example
def model_index_EO3_3(d1,d2,M):
  h1=convert_Imhash3(d2)
#   h1 = np.array(h1)
  for i1 in range(M):
    traind=globals()['x_hash%s' % i1]


    d=hamming_dist_2(h1[0],traind)
    member=np.count_nonzero(d < 14)
    if (member>=1):
      index=i1
      break
  if (member==0):
    index=M
  return index

# EO3_4 crop resistant hash function #


In [32]:
def convert_Imhash4(trainX1):
  lenth=len(trainX1)
  hashid=np.zeros(lenth)
  hash=[]
  for id in range(lenth):
    hashid[id]=id
    print(id)
    im_array=Image.fromarray(trainX1[id].astype('uint8'), 'RGB')

    hash1 = imagehash.crop_resistant_hash(im_array)

    hash.append(hash1)
  return np.array(hash)


In [33]:
#check which model contain the example
def model_index_EO3_4(d1,d2,M):
  for i1 in range(M):
    traind=globals()['x_hash%s' % i1]

    h1=convert_Imhash4(d2)
    d=hamming_dist(h1,traind)
    member=np.count_nonzero(d < 14)

    if (member>=1):
      index=i1
      break
  if (member==0):
    index=M
  return index

Exclusion Oracle-4:Classifier

Exclusion Oracle-5: Chain

transformation functions

In [34]:
from scipy import ndimage
def rotate_image(x1,a):  #angle
  x2=np.zeros(x1.shape)
  x3=np.zeros(x1.shape)
  tp=x1.dtype
  x2=x2.astype(tp)
  x3=x3.astype(tp)
  for i in range(len(x1)):
    out1=ndimage.rotate(x1[i],a,reshape=False)
    #out2=ndimage.rotate(x1[i],-a,reshape=False)
    x2[i]=out1
    #x3[i]=out2
  return x2
     

start evaluating PD attacks

In [35]:
EO

'EO3_1'

In [21]:
#calculate parameters from our technique: Only Model index for EO accuracy
#from joblib import Parallel, delayed
import time
def calc_EO_acc(data1,data2):
  if EO=='EO2':
    model_index=model_index_EO2
  elif EO=='EO3':
    model_index=model_index_EO3
  elif EO=='EO3_1':
    model_index=model_index_EO3_1
  elif EO=='EO4':
    model_index=model_index_EO4
  elif EO=='EO5':
    model_index=model_index_EO5
  
  n=len(data1)
  logits=np.zeros((n,num_class))
  prob=np.zeros((n,num_class))
  labels=np.zeros(n)
  index_resulted=np.zeros(n)
  ts=.02
  for p1 in range(n):
    kk=0
    d1=data1[p1:p1+1]
    d2=data2[p1:p1+1]
    #sum1=np.zeros(10)
    sum2=np.zeros(num_class)
    index=model_index(d1,d2,M)
    index_resulted[p1]=index
  return index_resulted

In [22]:
#calculate parameters from our combined
#from joblib import Parallel, delayed
import time
import timeit
def calc_attack_matrix(data1,data2):
  n=len(data1)
  logits=np.zeros((n,num_class))
  prob=np.zeros((n,num_class))
  labels=np.zeros(n)
  index_resulted=np.zeros(n)
  index_arr=np.zeros((n,293))
  ts=.02
  kk=0
  #Define Model Index
  if EO=='EO2':
    model_index=model_index_EO2
  elif EO=='EO3':
    model_index=model_index_EO3
  elif EO=='EO3_1':
    model_index=model_index_EO3_1  
  elif EO=='EO3_2':
    model_index=model_index_EO3_2  
  elif EO=='EO3_3':
    model_index=model_index_EO3_3  
  elif EO=='EO4':
    model_index=model_index_EO4
  elif EO=='EO5':
    model_index=model_index_EO5
  for p1 in range(n):
    d1=data1[p1:p1+1]
    d2=data2[p1:p1+1]
    #sum1=np.zeros(10)
    sum2=np.zeros(num_class)
    index=model_index(d1,d2,M)
    #print(arr_t.shape)
    #index_arr[p1,:]=arr_t
    index_resulted[p1]=index
    #print(index)
    for p2 in range(M):
      if(index!=p2):
        start = timeit.default_timer()
        modelh=globals()['model_%s' % p2]
        prob_d = modelh.predict(d1)
        #prob_d = special.softmax(logits_d)
        kk=kk+1
        #sum1=logits_d+sum1
        prob_vote=confidence(prob_d)
        sum2=prob_d+sum2
        #print(sum1)
        #print(sum2)
        stop = timeit.default_timer()
        ts=stop-start
      else:
        time.sleep(ts)
    #logits[p1,:]=sum1/100
    #print(logits)
    prob[p1,:]=sum2/kk*2
    #labels[p1,:]=np.argmax(logits[p1,:])
  return prob,index_resulted

Probability Dependent Attack

GAP Attack

Label-Only Attack:Initialization

RA attack

calculate the EO accuracy

# IMPORTANT VARIABLES #

In [23]:
from scipy import ndimage, misc
import matplotlib.pyplot as plt
dn=500

#Scaled /255 data
train1=x_train_split0[0:dn]
#Original image data
trainx1=x_traind0[0:dn]
#train labels from split 0
yt=trainy_split0[0:dn]
#test labels
ys=test_labels[5000:5000+dn]

#Scaled and original test data
test1=test_data[5000:5000+dn]
testx1=testX1[5000:5000+dn]
#Rotation angle
r=10

In [24]:
def rotate_image(x1,a):  #angle
  x2=np.zeros(x1.shape)
  x3=np.zeros(x1.shape)
  tp=x1.dtype
  x2=x2.astype(tp)
  x3=x3.astype(tp)
  for i in range(len(x1)):
    out1=ndimage.rotate(x1[i],a,reshape=False)
    out2=ndimage.rotate(x1[i],-a,reshape=False)
    x2[i]=out1
    x3[i]=out2
  return x2,x3

In [25]:
train_data_rotate1,train_data_rotate2=rotate_image(train_data[5000:5000+dn],r)
test_data_rotate1,test_data_rotate2=rotate_image(test_data[0:dn],r)

In [26]:
def calc_b(target_model,data,t):
  n=len(data)
  ytr=target_model.predict(data)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr,ytr

In [27]:
t=train_labels[5000:5000+dn].reshape(dn,1)
s=test_labels[0:dn].reshape(dn,1)
T=train_data[5000:5000+dn].reshape(dn,32*32*3)
S=test_data[0:dn].reshape(dn,32*32*3)

In [28]:
# r=10   #1-15
target_model=model_np 
private_model=model_np#the model that needs to be evaluated
# train=train_data[5000:10000]
# test=test_data[0:5000]

# ytr is model predictions #
# btr is true predictions #
length is same as input data length

rotate1 is + angle, rotate2 is -

In [29]:
#calculate b
btr1,ytr1=calc_b(target_model,train_data_rotate1,t)
btr2,ytr2=calc_b(target_model,train_data_rotate2,t)
btr3,ytr3=calc_b(target_model,train1,t)


bsr1,ysr1=calc_b(target_model,test_data_rotate1,s)
bsr2,ysr2=calc_b(target_model,test_data_rotate2,s)
bsr3,ysr3=calc_b(target_model,test1,s)

out1=np.ones(dn)
out2=np.zeros(dn)
label1=np.hstack((out1,out2))
print(label1.shape)
#rearrange b
btr=np.hstack((btr3,btr1,btr2))
#btr=np.transpose(btr)
#print(btr.shape)
bsr=np.hstack((bsr3,bsr1,bsr2))
#bsr=np.transpose(bsr)
b=np.vstack((btr,bsr))
b.shape

16/16 [==============================] - 0s 7ms/step
(1000,)


(1000, 3)

# t is train label, s is test labels #

In [30]:
total=np.concatenate([t, s])
Bt=np.hstack((b,total))
Bt.shape
label=label1.flatten()
#label=to_categorical(label1)

In [31]:
Bt.shape

(1000, 4)

For accuracy
# Bt is col1 = no rotation -train and test data true value array
# col2 = train data + rotatation true value array AND test data + rotatation true value array
# col3 = train data - rotatation true value array AND test data - rotatation true value array
# col4 = True labels of train and test


Basically training a small NN to predict from 3 MODIFIED datasets (+/- and no rotation) to true labels of ORIGINAL data
Accuracy of this NN model gives us an idea of how accurate the attack was

In [32]:
from sklearn.model_selection import train_test_split
t_data,s_data,t_label,s_label=train_test_split(Bt, label, test_size=0.1, random_state=42)

In [33]:
def make_fc():
  """Return conv_model as tf.keras.Model, without Softmax layer (only logits).
  To perform
  :param input_shape:
  :param depth:
  :param regularization:
  :param reg_constant:
  :param num_classes:
  :return:
  """
  fc_model = tf.keras.models.Sequential()
  #fc_model.add(tf.keras.layers.Dense(num_classes, kernel_regularizer=k_reg))
  fc_model.add(tf.keras.layers.Dense(10, activation=tf.keras.layers.ReLU(
        negative_slope=1e-2), kernel_initializer='glorot_normal'))
  fc_model.add(keras.layers.Dense(10, activation=tf.keras.layers.ReLU(
        negative_slope=1e-2), kernel_initializer='glorot_normal'))
  fc_model.add(tf.keras.layers.Dense(2, kernel_initializer='glorot_normal'))
  fc_model.add(tf.keras.layers.Softmax())
  return fc_model

In [46]:
#train a model
modelLB=make_fc()
optimizer = tf.keras.optimizers.Adam(learning_rate=.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# compile the model
modelLB.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# train the model
history = modelLB.fit(t_data, t_label,
                       validation_data=(s_data, s_label),
                       batch_size=32, 
                       epochs=30)

Epoch 1/30


C:\Users\neelk\anaconda3\envs\solo2\lib\site-packages\keras\backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


29/29 [==============================] - 1s 14ms/step - loss: 1.4318 - accuracy: 0.4967 - val_loss: 0.9382 - val_accuracy: 0.4500
Epoch 2/30
29/29 [==============================] - 0s 4ms/step - loss: 0.7836 - accuracy: 0.4856 - val_loss: 0.7259 - val_accuracy: 0.4500
Epoch 3/30
29/29 [==============================] - 0s 4ms/step - loss: 0.7006 - accuracy: 0.5089 - val_loss: 0.7224 - val_accuracy: 0.4500
Epoch 4/30
29/29 [==============================] - 0s 4ms/step - loss: 0.6982 - accuracy: 0.5100 - val_loss: 0.7186 - val_accuracy: 0.4400
Epoch 5/30
29/29 [==============================] - 0s 4ms/step - loss: 0.6969 - accuracy: 0.5178 - val_loss: 0.7171 - val_accuracy: 0.4500
Epoch 6/30
29/29 [==============================] - 0s 4ms/step - loss: 0.6947 - accuracy: 0.5122 - val_loss: 0.7142 - val_accuracy: 0.3900
Epoch 7/30
29/29 [==============================] - 0s 4ms/step - loss: 0.6954 - accuracy: 0.5056 - val_loss: 0.7124 - val_accuracy: 0.4500
Epoch 8/30
29/29 [============

In [47]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(s_data)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
accuracy_score(mem,s_label)

4/4 [==============================] - 0s 3ms/step


0.43

In [52]:
#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, s_label, pos_label=1)
metrics.auc(fpr, tpr)

0.45044413277232354

In [53]:
np.max(abs(fpr-tpr))

0.09911173445535293

In [54]:
#calculate train accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
accuracy_score(t,ytr1)

0.622

In [55]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
accuracy_score(s,ysr1)

0.558

In [274]:
# stop!!

# Private Model Attacks #

In [35]:
def calc_b(private_model,data1,data2,t):    #if only model_stacking
  n=len(data1)
  ytr,indx=calc_attack_matrix(data1,data2)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr,ytr

In [36]:
#create training and testing rotate data
train_data_rotateT1,train_data_rotateT2=rotate_image(train1,r)
train_X1_rotateT1,train_X1_rotateT2=rotate_image(trainx1,r)
test_data_rotateT1,test_data_rotateT2=rotate_image(test1,r)
test_X1_rotateT1,test_X1_rotateT2=rotate_image(testx1,r)

In [37]:
t=yt.reshape(dn,1)
s=ys.reshape(dn,1)
T=train1.reshape(dn,32*32*3)
S=test1.reshape(dn,32*32*3)

In [45]:
test1.shape

(500, 32, 32, 3)

# RA Attack on ASE EO AHASH #

In [46]:
EO= 'EO3_1'

In [ ]:
#calculate and store hashes for M=5 #different model and their training dataset
#Only for imagehash
for x in range(0,M):
             #print(j,k+j)
             x1=globals()['x_traind%s' % x]
             x2=convert_Imhash1(x1)
             globals()['x_hash%s' % x]=x2
             #print(globals()['x_hash%s' % x])

In [151]:
#calculate b
Btr1,Ytr1=calc_b(private_model,train_data_rotateT1,train_X1_rotateT1,t)
Btr2,Ytr2=calc_b(private_model,train_data_rotateT2,train_X1_rotateT2,t)
Btr3,Ytr3=calc_b(private_model,train1,trainx1,t)

Bsr1,Ysr1=calc_b(private_model,test_data_rotateT1,test_X1_rotateT1,s)
Bsr2,Ysr2=calc_b(private_model,test_data_rotateT2,test_X1_rotateT2,s)
Bsr3,Ysr3=calc_b(private_model,test1,testx1,s)

out1=np.ones(dn)
out2=np.zeros(dn)
label1=np.hstack((out1,out2))
print(label1.shape)
#rearrange b
Btr=np.hstack((Btr3,Btr1,Btr2))
#btr=np.transpose(btr)
#print(btr.shape)
Bsr=np.hstack((Bsr3,Bsr1,Bsr2))
#bsr=np.transpose(bsr)
B=np.vstack((Btr,Bsr))
B.shape

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 20ms/step


...

1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 18ms/step
(1000,)


(1000, 3)

In [152]:
total=np.concatenate([t, s])
Btotal=np.hstack((B,total))
Btotal.shape
label=label1.flatten()
#label=to_categorical(label1)

# Attack accuracy #

In [153]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(Btotal)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
accuracy_score(mem,label)

32/32 [==============================] - 0s 704us/step


0.497

In [154]:
#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, label, pos_label=1)
metrics.auc(fpr, tpr)

0.4969776466748069

In [155]:
np.max(abs(fpr-tpr))

0.0060447066503862

In [156]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
accuracy_score(t,Ytr1)

0.566

In [157]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
accuracy_score(s,Ysr1)

0.526

# RA Attack on ASE EO AHASH + DHASH #

In [272]:
EO = 'EO3_2'

In [248]:
#calculate and store hashes for M=5 #different model and their training dataset
#Only for imagehash
for x in range(0,M):
             #print(j,k+j)
             x1=globals()['x_traind%s' % x]
             x2=convert_Imhash2(x1)
             globals()['x_hash%s' % x]=x2
             #print(globals()['x_hash%s' % x])

In [273]:
#calculate b
Btr1,Ytr1=calc_b(private_model,train_data_rotateT1,train_X1_rotateT1,t)
Btr2,Ytr2=calc_b(private_model,train_data_rotateT2,train_X1_rotateT2,t)
Btr3,Ytr3=calc_b(private_model,train1,trainx1,t)

Bsr1,Ysr1=calc_b(private_model,test_data_rotateT1,test_X1_rotateT1,s)
Bsr2,Ysr2=calc_b(private_model,test_data_rotateT2,test_X1_rotateT2,s)
Bsr3,Ysr3=calc_b(private_model,test1,testx1,s)

out1=np.ones(dn)
out2=np.zeros(dn)
label1=np.hstack((out1,out2))
print(label1.shape)
#rearrange b
Btr=np.hstack((Btr3,Btr1,Btr2))
#btr=np.transpose(btr)
#print(btr.shape)
Bsr=np.hstack((Bsr3,Bsr1,Bsr2))
#bsr=np.transpose(bsr)
B=np.vstack((Btr,Bsr))
B.shape

1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 26ms/step


...

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 12ms/step
(1000,)


(1000, 3)

In [275]:
total=np.concatenate([t, s])
Btotal=np.hstack((B,total))
Btotal.shape
label=label1.flatten()
#label=to_categorical(label1)

In [276]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(Btotal)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
print('\n Attack accuracy')
print(accuracy_score(mem,label))


#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, label, pos_label=1)
print('\n Area under attack curve')
print(metrics.auc(fpr, tpr))

print('\n Attack advantage')
print(np.max(abs(fpr-tpr)))

#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
print('\n Train accuracy ', accuracy_score(t,Ytr1))

#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
print('\n Test accuracy ', accuracy_score(s,Ysr1))

32/32 [==============================] - 0s 1ms/step

 Attack accuracy
0.485

 Area under attack curve
0.4848775078132876

 Attack advantage
0.030244984373424766

 Train accuracy  0.566

 Test accuracy  0.536


# RA Attack on ASE EO Crop resistant hash #

In [ ]:
EO = 'EO3_4'

In [ ]:
#calculate and store hashes for M=5 #different model and their training dataset
#Only for imagehash
for x in range(0,M):
             #print(j,k+j)
             x1=globals()['x_traind%s' % x]
             x2=convert_Imhash4(x1)
             globals()['x_hash%s' % x]=x2
             #print(globals()['x_hash%s' % x])

In [ ]:
#calculate b
Btr1,Ytr1=calc_b(private_model,train_data_rotateT1,train_X1_rotateT1,t)
Btr2,Ytr2=calc_b(private_model,train_data_rotateT2,train_X1_rotateT2,t)
Btr3,Ytr3=calc_b(private_model,train1,trainx1,t)

Bsr1,Ysr1=calc_b(private_model,test_data_rotateT1,test_X1_rotateT1,s)
Bsr2,Ysr2=calc_b(private_model,test_data_rotateT2,test_X1_rotateT2,s)
Bsr3,Ysr3=calc_b(private_model,test1,testx1,s)

out1=np.ones(dn)
out2=np.zeros(dn)
label1=np.hstack((out1,out2))
print(label1.shape)
#rearrange b
Btr=np.hstack((Btr3,Btr1,Btr2))
#btr=np.transpose(btr)
#print(btr.shape)
Bsr=np.hstack((Bsr3,Bsr1,Bsr2))
#bsr=np.transpose(bsr)
B=np.vstack((Btr,Bsr))
B.shape

In [ ]:
total=np.concatenate([t, s])
Btotal=np.hstack((B,total))
Btotal.shape
label=label1.flatten()
#label=to_categorical(label1)

In [ ]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(Btotal)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
print('\n Attack accuracy')
print(accuracy_score(mem,label))


#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, label, pos_label=1)
print('\n Area under attack curve')
print(metrics.auc(fpr, tpr))

print('\n Attack advantage')
print(np.max(abs(fpr-tpr)))

#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
print('\n Train accuracy ', accuracy_score(t,Ytr1))

#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
print('\n Test accuracy ', accuracy_score(s,Ysr1))

# RA Attack on ASE EO BHASH #

In [38]:
EO = 'EO3_3'

In [39]:
#calculate and store hashes for M=5 #different model and their training dataset
#Only for imagehash
for x in range(0,M):
             #print(j,k+j)
             x1=globals()['x_traind%s' % x]
             x2=convert_Imhash3(x1)
             globals()['x_hash%s' % x]=x2
             #print(globals()['x_hash%s' % x])

NameError: name 'convert_Imhash3' is not defined

In [40]:
#calculate b
Btr1,Ytr1=calc_b(private_model,train_data_rotateT1,train_X1_rotateT1,t)
Btr2,Ytr2=calc_b(private_model,train_data_rotateT2,train_X1_rotateT2,t)
Btr3,Ytr3=calc_b(private_model,train1,trainx1,t)

Bsr1,Ysr1=calc_b(private_model,test_data_rotateT1,test_X1_rotateT1,s)
Bsr2,Ysr2=calc_b(private_model,test_data_rotateT2,test_X1_rotateT2,s)
Bsr3,Ysr3=calc_b(private_model,test1,testx1,s)

out1=np.ones(dn)
out2=np.zeros(dn)
label1=np.hstack((out1,out2))
print(label1.shape)
#rearrange b
Btr=np.hstack((Btr3,Btr1,Btr2))
#btr=np.transpose(btr)
#print(btr.shape)
Bsr=np.hstack((Bsr3,Bsr1,Bsr2))
#bsr=np.transpose(bsr)
B=np.vstack((Btr,Bsr))
B.shape

NameError: name 'model_index_EO3_3' is not defined

In [73]:
total=np.concatenate([t, s])
Btotal=np.hstack((B,total))
Btotal.shape
label=label1.flatten()
#label=to_categorical(label1)

In [74]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(Btotal)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
print('\n Attack accuracy')
print(accuracy_score(mem,label))


#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, label, pos_label=1)
print('\n Area under attack curve')
print(metrics.auc(fpr, tpr))

print('\n Attack advantage')
print(np.max(abs(fpr-tpr)))

#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
print('\n Train accuracy ', accuracy_score(t,Ytr1))

#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
print('\n Test accuracy ', accuracy_score(s,Ysr1))

32/32 [==============================] - 0s 1ms/step

 Attack accuracy
0.514

 Area under attack curve
0.5155530670648252

 Attack advantage
0.031106134129650354

 Train accuracy  0.56

 Test accuracy  0.532


# RA Attack on ASE EO PHASH #


In [41]:
EO = 'EO3'

In [42]:
#calculate and store hashes for M=5 #different model and their training dataset
#Only for imagehash
for x in range(0,M):
             #print(j,k+j)
             x1=globals()['x_traind%s' % x]
             x2=convert_Imhash(x1)
             globals()['x_hash%s' % x]=x2
             #print(globals()['x_hash%s' % x])

In [43]:
#calculate b
Btr1,Ytr1=calc_b(private_model,train_data_rotateT1,train_X1_rotateT1,t)
Btr2,Ytr2=calc_b(private_model,train_data_rotateT2,train_X1_rotateT2,t)
Btr3,Ytr3=calc_b(private_model,train1,trainx1,t)

Bsr1,Ysr1=calc_b(private_model,test_data_rotateT1,test_X1_rotateT1,s)
Bsr2,Ysr2=calc_b(private_model,test_data_rotateT2,test_X1_rotateT2,s)
Bsr3,Ysr3=calc_b(private_model,test1,testx1,s)

out1=np.ones(dn)
out2=np.zeros(dn)
label1=np.hstack((out1,out2))
print(label1.shape)
#rearrange b
Btr=np.hstack((Btr3,Btr1,Btr2))
#btr=np.transpose(btr)
#print(btr.shape)
Bsr=np.hstack((Bsr3,Bsr1,Bsr2))
#bsr=np.transpose(bsr)
B=np.vstack((Btr,Bsr))
B.shape

1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 20ms/step


...

1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 18ms/step
(1000,)


(1000, 3)

In [44]:
total=np.concatenate([t, s])
Btotal=np.hstack((B,total))
Btotal.shape
label=label1.flatten()
#label=to_categorical(label1)

In [45]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(Btotal)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
print('\n Attack accuracy')
print(accuracy_score(mem,label))


#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, label, pos_label=1)
print('\n Area under attack curve')
print(metrics.auc(fpr, tpr))

print('\n Attack advantage')
print(np.max(abs(fpr-tpr)))

#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
print('\n Train accuracy ', accuracy_score(t,Ytr1))

#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
print('\n Test accuracy ', accuracy_score(s,Ysr1))

32/32 [==============================] - 0s 705us/step

 Attack accuracy
0.502

 Area under attack curve
0.502241107286288

 Attack advantage
0.0044822145725760265

 Train accuracy  0.568

 Test accuracy  0.53


In [ ]:
stop!!

TA attack

In [ ]:
#d=1
N=5
#d=np.array([0,1,-1,1,-1]) 
#d=np.array([0,3,2,1,-3,-2,-1,3,2,1,-3,-2,-1])  #d=3
d=np.array([0,5,4,3,2,1,-5,-4,-3,-2,-1,5,4,3,2,1,-5,-4,-3,-2,-1])
kv=np.ceil(len(d)/2)

In [ ]:
import random
def translate_images1(x1,d):  #d=distance
  x2=np.zeros(x1.shape)
  x3=np.zeros(x1.shape)
  tp=x1.dtype
  x2=x2.astype(tp)
  x3=x3.astype(tp)
  for i in range(len(x1)):
    test_x=x1[i]
    if d==0:
      i1=0
      j1=0
    else:
      i1=d#random.uniform(-d,d)
      j1=N-abs(i1)

    out1=ndimage.interpolation.shift(test_x,np.array([i1,j1,0,]))
    #print(i1,j1)
    #i2=0#random.uniform(-d,d)
    #j2=d
    #out2=ndimage.interpolation.shift(test_x,np.array([i2,j2,0]))
    
    #_____________________________________________#
    x2[i]=out1
    

  return x2

In [ ]:
import random
def translate_images2(x1,d):  #d=distance
  x2=np.zeros(x1.shape)
  x3=np.zeros(x1.shape)
  tp=x1.dtype
  x2=x2.astype(tp)
  x3=x3.astype(tp)
  for i in range(len(x1)):
    test_x=x1[i]
    j1=d#random.uniform(-d,d)
    i1=N-abs(j1)
    out1=ndimage.interpolation.shift(test_x,np.array([i1,j1,0]))
    #print(i1,j1)
    #i2=0#random.uniform(-d,d)
    #j2=d
    #out2=ndimage.interpolation.shift(test_x,np.array([i2,j2,0]))
    
    #_____________________________________________#
    x2[i]=out1
    

  return x2

In [ ]:
def calc_b(target_model,data,t,d,k):
  n=len(data)
  if k>=kv:
    translate_images=translate_images2
  else:
    translate_images=translate_images1
  datat=translate_images(data,d)
  ytr=target_model.predict(datat)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  ytr=ytr.reshape(len(ytr))
  btr=btr.reshape(len(btr))
  return btr,ytr

In [ ]:
t=train_labels[5000:10000].reshape(5000,1)
s=test_labels[0:5000].reshape(5000,1)
T=train_data[5000:10000].reshape(5000,32*32*3)
S=test_data[0:5000].reshape(5000,32*32*3)

In [ ]:
btr=np.zeros((len(t),len(d)))
ytr=np.zeros((len(t),len(d)))
#indxt=np.zeros(len(t),len(d))
print('Calculate features for training data')
for i in range(len(d)):
  btr[:,i],ytr[:,i]=calc_b(target_model,train_data[5000:10000],t,d[i],i)

print('Calculate features for testing data')
bsr=np.zeros((len(t),len(d)))
ysr=np.zeros((len(t),len(d)))
#indxt=np.zeros(len(t),len(d))
for i in range(len(d)):
  bsr[:,i],ysr[:,i]=calc_b(target_model,test_data[0:5000],s,d[i],i)


print('rearrange b')
b=np.vstack((btr,bsr))
b.shape

In [ ]:
total=np.concatenate([t, s])
B=np.hstack((b,total))
B.shape

In [ ]:
out1=np.ones(dn)
out2=np.zeros(dn)
label1=np.hstack((out1,out2))
print(label1.shape)
label=label1.flatten()
#label=to_categorical(label1)

(10000,)


In [ ]:
from sklearn.model_selection import train_test_split
t_data,s_data,t_label,s_label=train_test_split(B, label, test_size=0.2, random_state=42)

In [ ]:
#train a model
modelLB=make_fc()
optimizer = tf.keras.optimizers.Adam(learning_rate=.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# compile the model
modelLB.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# train the model
history = modelLB.fit(t_data, t_label,
                       validation_data=(s_data, s_label),
                       batch_size=32, 
                       epochs=30)

In [ ]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(s_data)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
accuracy_score(mem,s_label)

In [ ]:
#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, s_label, pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
np.max(abs(fpr-tpr))

In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
#acct=accuracy_score(t,ytr1)+accuracy_score(t,ytr2)+accuracy_score(t,ytr3)+accuracy_score(t,ytr4)+accuracy_score(t,ytr5)
#print(acct/5)
acct=np.zeros(len(d))
for i in range(len(d)):
  acct[i]=accuracy_score(t,ytr[:,i])

print(np.average(acct))

0.8135999999999999


In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
#accs=accuracy_score(s,ysr1)+accuracy_score(s,ysr1)+accuracy_score(s,ysr1)+accuracy_score(s,ysr1)+accuracy_score(s,ysr1)
#print(accs/5)
accs=np.zeros(len(d))
for i in range(len(d)):
  accs[i]=accuracy_score(s,ysr[:,i])

print(np.average(accs))

0.659


In [ ]:
stop!!

Private Settings

In [ ]:
def calc_B(target_model,data1,data2,t,d,k):
  n=len(data1)
  if k>=kv:
    translate_images=translate_images2
  else:
    translate_images=translate_images1
  datat1=translate_images(data1,d)
  datat2=translate_images(data2,d)
  ytr,indx=calc_attack_matrix(datat1,datat2)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  ytr=ytr.reshape(len(ytr))
  btr=btr.reshape(len(btr))
  indx=indx.reshape(len(indx))
  return btr,ytr,indx

In [ ]:
t=yt.reshape(dn,1)
s=ys.reshape(dn,1)
T=train1.reshape(dn,32*32*3)
S=test1.reshape(dn,32*32*3)

In [ ]:
btr=np.zeros((len(t),len(d)))
ytr=np.zeros((len(t),len(d)))
indxt=np.zeros((len(t),len(d)))
print('Calculate features for training data')
for i in range(len(d)):
  btr[:,i],ytr[:,i],indxt[:,i]=calc_B(target_model,train1,trainx1,t,d[i],i)

print('Calculate features for testing data')
bsr=np.zeros((len(t),len(d)))
ysr=np.zeros((len(t),len(d)))
indxs=np.zeros((len(t),len(d)))
for i in range(len(d)):
  bsr[:,i],ysr[:,i],indxs[:,i]=calc_B(target_model,test1,testx1,s,d[i],i)


print('rearrange b')
b1=np.vstack((btr,bsr))
b1.shape

In [ ]:
total=np.concatenate([t, s])
Btotal=np.hstack((b1,total))
Btotal.shape

(5000, 6)

In [ ]:
out1=np.ones(dn)
out2=np.zeros(dn)
label1=np.hstack((out1,out2))
print(label1.shape)
label=label1.flatten()
#label=to_categorical(label1)

(5000,)


In [ ]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(Btotal)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
accuracy_score(mem,label)

0.5114

In [ ]:
#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, label, pos_label=1)
metrics.auc(fpr, tpr)

0.5134056807348233

In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
#acct=accuracy_score(t,ytr1)+accuracy_score(t,ytr2)+accuracy_score(t,ytr3)+accuracy_score(t,ytr4)+accuracy_score(t,ytr5)
#print(acct/5)
acct=np.zeros(len(d))
for i in range(len(d)):
  acct[i]=accuracy_score(t,ytr[:,i])

print(np.average(acct))

0.69104


In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
#acct=accuracy_score(t,ytr1)+accuracy_score(t,ytr2)+accuracy_score(t,ytr3)+accuracy_score(t,ytr4)+accuracy_score(t,ytr5)
#print(acct/5)
accs=np.zeros(len(d))
for i in range(len(d)):
  accs[i]=accuracy_score(s,ysr[:,i])

print(np.average(accs))

0.6668


In [ ]:
np.max(abs(fpr-tpr))

0.026811361469646466

In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
#acct=accuracy_score(t,ytr1)+accuracy_score(t,ytr2)+accuracy_score(t,ytr3)+accuracy_score(t,ytr4)+accuracy_score(t,ytr5)
#print(acct/5)
accindx=np.zeros(len(d))
for i in range(len(d)):
  result_index=np.hstack((indxt[:,i],indxs[:,i]))
  accindx[i]=accuracy_score(index_target,result_index)

print(np.average(accindx))

In [ ]:
stop!!

Boundary Distance Attack

In [44]:
n1=500

In [45]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve

In [46]:
def continuous_rand_robust(model, dsx,dsy, max_samples=100, noise_samples=250, stddev=0.025, input_dim=shape,
                           num=[1, 2, 3, 4, 5, 10, 20, 30, 40, 50, 75, 100, 150, 250]): #350, 500, 750, 1000, 1500, 2000, 2500]):
  """Calculate robustness to random noise for Adv-x MI attack on continuous-featureed datasets (+ UCI adult).
  :param model: model to approximate distances on (attack).
  :param ds: tf dataset should be either the training set or the test set.
  :param max_samples: maximum number of samples to take from the ds
  :param noise_samples: number of noised samples to take for each sample in the ds.
  :param stddev: the standard deviation to use for Gaussian noise (only for Adult, which has some continuous features)
  :param input_dim: dimension of inputs for the dataset.
  :param num: subnumber of samples to evaluate. max number is noise_samples
  :return: a list of lists. each sublist of the accuracy on up to $num noise_samples.
  """
  robust_accs = [[] for _ in num]
  iter=len(dsx)
  labels=dsy
  for i in range(iter):
    x=dsx[i:i+1]
    noise = stddev * np.random.randn(noise_samples, 32,32,3)
    x_noisy = np.clip(x + noise, 0, 1)
    #print(noise.shape)
    prob=model.predict(x_noisy)
    pred=np.argmax(prob,axis=1)
    #print(pred.shape)
    for idx, n in enumerate(num):
            if n == 0:
              robust_accs[idx].append(1)
            else:
              robust_accs[idx].append(np.mean(pred[:n] == labels[i]))
  return robust_accs

In [47]:
def continuous_rand_robust_target(model, dsx,dsy,dX, max_samples=100, noise_samples=250, stddev=0.025, input_dim=shape,
                           num=[1, 2, 3, 4, 5, 10, 20,30, 40, 50, 75, 100, 150, 250]):#350, 500, 750, 1000, 1500, 2000, 2500]):
  """Calculate robustness to random noise for Adv-x MI attack on continuous-featureed datasets (+ UCI adult).
  :param model: model to approximate distances on (attack).
  :param ds: tf dataset should be either the training set or the test set.
  :param max_samples: maximum number of samples to take from the ds
  :param noise_samples: number of noised samples to take for each sample in the ds.
  :param stddev: the standard deviation to use for Gaussian noise (only for Adult, which has some continuous features)
  :param input_dim: dimension of inputs for the dataset.
  :param num: subnumber of samples to evaluate. max number is noise_samples
  :return: a list of lists. each sublist of the accuracy on up to $num noise_samples.
  """
  robust_accs = [[] for _ in num]
  iter=len(dsx)
  labels=dsy
  for i in range(iter):
    x=dsx[i:i+1]
    X=dX[i:i+1]
    noise = stddev * np.random.randn(noise_samples, 32,32,3)
    x_noisy = np.clip(x + noise, 0, 1)
    X_noisy=np.clip(X + noise, 0, 255)
    #print(noise.shape)
    print('error1')
    prob,indx=calc_attack_matrix(x_noisy,X_noisy)
    pred=np.argmax(prob,axis=1)
    print('error2')
    #print(pred)
    for idx, n in enumerate(num):
            if n == 0:
              robust_accs[idx].append(1)
            else:
              robust_accs[idx].append(np.mean(pred[:n] == labels[i]))
  return robust_accs

non-Private

In [48]:
noise_source_in=continuous_rand_robust(target_model,train_data[0:n1],train_labels[0:n1])
noise_source_out=continuous_rand_robust(target_model,test_data[0:n1],test_labels[0:n1])

8/8 [==============================] - 0s 6ms/step


8/8 [==============================] - 0s 9ms/step


8/8 [==============================] - 0s 8ms/step


8/8 [==============================] - 0s 8ms/step


private

In [53]:
import timeit
noise_target_in=continuous_rand_robust_target(model_np,train_data[n1:n1*2],train_labels[n1:n1*2],trainX1[n1:n1*2])
noise_target_out=continuous_rand_robust_target(model_np,test_data[n1:n1*2],test_labels[n1:n1*2],testX1[n1:n1*2])

error1


NameError: name 'calc_attack_matrix' is not defined

In [49]:
source_m = np.concatenate([np.ones(n1),np.zeros(n1)], axis=0)
target_m = np.concatenate([np.ones(n1),np.zeros(n1)], axis=0)

In [50]:
def get_max_accuracy(y_true, probs, thresholds=None):
  """Return the max accuracy possible given the correct labels and guesses. Will try all thresholds unless passed.
  Args:
    y_true: True label of `in' or `out' (member or non-member, 1/0)
    probs: The scalar to threshold
    thresholds: In a blackbox setup with a shadow/source model, the threshold obtained by the source model can be passed
      here for attackin the target model. This threshold will then be used.
  Returns: max accuracy possible, accuracy at the threshold passed (if one was passed), the max precision possible,
   and the precision at the threshold passed.
  """
  attack_adv=[]
  if thresholds is None:
    fpr, tpr, thresholds = roc_curve(y_true, probs)
    attack_adv.append(abs(tpr-fpr))
    

  accuracy_scores = []
  precision_scores = []
  for thresh in thresholds:
    accuracy_scores.append(accuracy_score(y_true,
                                          [1 if m > thresh else 0 for m in probs]))
    precision_scores.append(precision_score(y_true, [1 if m > thresh else 0 for m in probs]))

  accuracies = np.array(accuracy_scores)
  precisions = np.array(precision_scores)
  attack_adv=np.array(attack_adv)
  max_accuracy = accuracies.max()
  max_precision = precisions.max()
  #print(type(attack_adv))
  #print(attack_adv)
  if (attack_adv.size !=0):
      max_adv=np.max(attack_adv)
      print(max_adv)
  max_accuracy_threshold = thresholds[accuracies.argmax()]
  max_precision_threshold = thresholds[precisions.argmax()]
  return max_accuracy, max_accuracy_threshold, max_precision, max_precision_threshold

In [51]:
def get_threshold(source_m, source_stats, target_m, target_stats):
  """ Train a threshold attack model and get teh accuracy on source and target models.
  Args:
    source_m: membership labels for source dataset (1 for member, 0 for non-member)
    source_stats: scalar values to threshold (attack features) for source dataset
    target_m: membership labels for target dataset (1 for member, 0 for non-member)
    target_stats: scalar values to threshold (attack features) for target dataset
  Returns: best acc from source thresh, precision @ same threshold, threshold for best acc,
    precision at the best threshold for precision. all tuned on source model.
  """
  # find best threshold on source data
  acc_source, t, prec_source, tprec = get_max_accuracy(source_m, source_stats)

  # find best accuracy on test data (just to check how much we overfit)
  acc_test, _, prec_test, _ = get_max_accuracy(target_m, target_stats)

  # get the test accuracy at the threshold selected on the source data
  acc_test_t, _, _, _ = get_max_accuracy(target_m, target_stats, thresholds=[t])
  _, _, prec_test_t, _ = get_max_accuracy(target_m, target_stats, thresholds=[tprec])
  print("acc src: {}, acc test (best thresh): {}, acc test (src thresh): {}, thresh: {}".format(acc_source, acc_test,
                                                                                                acc_test_t, t))
  print(
    "prec src: {}, prec test (best thresh): {}, prec test (src thresh): {}, thresh: {}".format(prec_source, prec_test,
                                                                                               prec_test_t, tprec))

  return acc_test_t, prec_test_t, t, tprec

In [52]:
for i in range(len(noise_source_in)):
            noise_source = np.concatenate([noise_source_in[i], noise_source_out[i]], axis=0)
            noise_target = np.concatenate([noise_target_in[i], noise_target_out[i]], axis=0)
            get_threshold(source_m, noise_source, target_m, noise_target)

NameError: name 'noise_target_in' is not defined